========================================
   
__コンテンツ__
* サーチ方法
   1. モジュールインポートとデータロード
   2. パラメータサーチ範囲の設定
   3. 特徴量選択範囲の設定 (optional)
   4. サーチの実行
   
* ログ使用方法
   1. サーチ結果の抽出
   2. スタッキングのためのメタ特徴量生成
   
========================================

# サーチ方法

## 1. モジュールインポートとデータロード
ここではサンプルデータとして"the breast cancer wisconsin dataset"（2値分類）を使用する。
はじめにデータセットをTrain, Testで分割する。

In [ ]:
import os ,sys
import numpy as np, pandas as pd, scipy as sp
from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression

from cvopt.model_selection import SimpleoptCV
from cvopt.search_setting import search_category, search_numeric

dataset = datasets.load_breast_cancer()
Xtrain, Xtest, ytrain, ytest = train_test_split(dataset.data, dataset.target, test_size=0.3, random_state=0)

print("Train features shape:", Xtrain.shape)
print("Test features shape:", Xtest.shape)

In [ ]:
from bokeh.io import output_notebook
output_notebook() # When you need search visualization, need run output_notebook()

## 2. パラメータサーチ範囲の設定
設定は、各cvクラス共通の書式で行うことが可能。

In [ ]:
param_distributions = {
    "penalty": search_category(['l1', 'l2']),
    "C": search_numeric(0.01, 3.0, "float"), 
    "tol" : search_numeric(0.0001, 0.001, "float"),  
    "class_weight" : search_category([None, "balanced"]),
    }

### 2.A その他の書式
各cvクラスのベースモジュールと同様の方法の書式も使用可能。 

### for HyperoptCV (base module: Hyperopt)
```python
param_distributions = {
    "penalty": hp.choice("penalty", ["l1", "l2"]),
    "C": hp.loguniform("C", 0.01, 3.0), 
    "tol" : hp.loguniform("tol", 0.0001, 0.001), 
    "class_weight" : hp.choice("class_weight", [None, "balanced"]),
    }
```
### for  BayesoptCV (base module: GpyOpt)
__NOTE:__
* GpyOptでは、サーチ範囲を辞書のリストで設定するが、本モジュールでは辞書の辞書(key:param name, value:dict{GpyOpt標準のサーチ範囲設定辞書})で設定を行う。
* カテゴリカルのパラメータの場合、サーチ範囲設定辞書に key:`categories`, value:`カテゴリ名一覧リスト` を追加する必要がある。
```python
param_distributions = {
    "penalty" : {"name": "penalty", "type":"categorical", "domain":(0,1), "categories":["l1", "l2"]},
    "C": {"name": "C", "type":"continuous", "domain":(0.01, 3.0)}, 
    "tol" : {"name": "tol", "type":"continuous", "domain":(0.0001, 0.001)}, 
    "class_weight" : {"name": "class_weight", "type":"categorical", "domain":(0,1), "categories":[None, "balanced"]},
    }
```

### for  GASearchCV, RandomoptCV
__NOTE:__
* サポート対象は`search_setting.search_numeric`, `search_setting.search_category`, `scipy.stats`クラスになる。
```python
param_distributions = {
    "penalty" : hp.choice("penalty", ["l1", "l2"]),
    "C": sp.stats.randint(low=0.01, high=3.0), 
    "tol" : sp.stats.uniform(loc=0.0001, scale=0.00009),  
    "class_weight" :  hp.choice("class_weight", [None, "balanced"],
    }
```

## 3. 特徴量選択範囲の設定 (optional)
特徴量選択は`feature_group`毎に行う。   
__もし`feature_group`が"-100"であれば、そのグループの特徴量は必ず選択される。__   
グループを分ける方法は例えばランダム、特徴量エンジニアリング方法の違い、データソースの違いがある。
`feature_group`を設定しない場合、全ての特徴量が使用される。

------------------------------------

### Example.   
データが5個の特徴量（カラム）を持っていて、以下のように`feature_group`を設定するとする。
   
| feature index(data col index) | feature group |   
|:------------:|:------------:|   
| 0 | 0 |
| 1 | 0 |
| 2 | 0 |
| 3 | 1 |
| 4 | 1 |
   
この場合、以下のようにlistを定義する。   
   
```
feature_groups = [0, 0, 0, 1, 1]
```

サーチの結果として, `feature_group`毎に選択したかどうかを表すbooleanが得られる。
   
```
feature_groups0: True   
feature_groups1: False
```

この結果は、グループ0の特徴量を使用し、グループ1の特徴量を使用しないという意味になる。

------------------------------------


In [ ]:
feature_groups = np.random.randint(0, 5, Xtrain.shape[1]) 

## 4. サーチの実行
cvoptはscikit-learnのcross validationクラスと同様のAPIを持っている。 
scikit-learnを使い慣れていれば、cvoptのクラスも簡単に使用することが出来る。
   
各クラスの詳細は[API reference](https://genfifth.github.io/cvopt/)を参照のこと。

In [ ]:
estimator = LogisticRegression()
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

opt = SimpleoptCV(estimator, param_distributions, 
                 scoring="roc_auc",              # Objective of search
                 cv=cv,                          # Cross validation setting
                 max_iter=32,                    # Number of search
                 n_jobs=3,                       # Number of jobs to run in parallel.
                 verbose=2,                      # 0: don't display status, 1:display status by stdout, 2:display status by graph 
                 logdir="./search_usage",        # If this path is specified, save the log.
                 model_id="search_usage",        # used estimator's dir and file name in save.
                 save_estimator=2,               # estimator save setting.
                 backend="hyperopt",             # hyperopt,bayesopt, gaopt or randomopt.
                 )

opt.fit(Xtrain, ytrain, validation_data=(Xtest, ytest), 
        # validation_data is optional.
        # This data is only used to compute validation score(don't fit).
        # When this data is input & save_estimator=True,the estimator which is fitted whole Xtrain is saved.
        feature_groups=feature_groups, 
        )
ytest_pred = opt.predict(Xtest)

In [ ]:
pd.DataFrame(opt.cv_results_).head() # Search results

# ログ使用方法

## 1.サーチ結果の抽出
cvoptでは、サーチログを簡単に使用するためのヘルパー関数を用意している。
サーチ結果を使用したい場合、以下のように実行可能。

In [ ]:
from cvopt.utils import extract_params
estimator_params, feature_params, feature_select_flag  = extract_params(logdir="./search_usage", 
                                                                        model_id="search_usage", 
                                                                        target_index=0, 
                                                                        feature_groups=feature_groups)

estimator.set_params(**estimator_params)         # Set estimator parameters
Xtrain_selected = Xtrain[:, feature_select_flag] # Extract selected feature columns

print(estimator)
print("Train features shape:", Xtrain.shape)
print("Train selected features shape:",Xtrain_selected.shape)

## 2. スタッキングのためのメタ特徴量生成
メタ特徴量を使用し、 [stacking](https://mlwave.com/kaggle-ensembling-guide/)をしたい場合、以下のようにメタ特徴量を取得できる。
   
これを行う場合、サーチ時にパラメータ`save_estimator`を0より大きくすることで、各cv-foldを実施した際のestimatorを保存する必要がある。 
加えて、fitしていない特徴量に対するメタ特徴量を生成したい場合、パラメータ`save_estimator`を1より大きくすることで、全trainデータでfitしたestimatorを保存する必要がある。

In [ ]:
from cvopt.utils import mk_metafeature
Xtrain_meta, Xtest_meta = mk_metafeature(Xtrain, ytrain, 
                                         logdir="./search_usage", 
                                         model_id="search_usage", 
                                         target_index=0, 
                                         cv=cv, 
                                         validation_data=(Xtest, ytest), 
                                         feature_groups=feature_groups, 
                                         estimator_method="predict_proba")

print("Train features shape:", Xtrain.shape)
print("Train meta features shape:", Xtrain_meta.shape)
print("Test features shape:", Xtest.shape)
print("Test meta features shape:",  Xtest_meta.shape)